ED of batteries (Assume fully charged and ready to be dispatched)

Values in dataset are randomly generated. Needs to be fine-tuned, should you want model a realistic scenario. No code modification is needed. 

In [ ]:
# Installing pyomo and 'cbc' solver
!pip install pyomo
!apt-get install -y -qq coinor-cbc
from pyomo.environ import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Reading the data
battery_data = pd.read_excel('Energy_Storage_ED.xlsx', sheet_name='Battery_Data')
load_data = pd.read_excel('Energy_Storage_ED.xlsx', sheet_name='Demand_Data')

In [ ]:
# Converting atttributes to numpy arrays
discharge_unitary_cost = battery_data.loc[:, 'CD ($/MWh)'].to_numpy()
installed_energy_capacity = battery_data.loc[:, 'SE (MWh)'].to_numpy()
rated_power = battery_data.loc[:, 'SP (MW)'].to_numpy()
discharge_rate = battery_data.loc[:, 'MD'].to_numpy()
discharge_efficiency = battery_data.loc[:, 'EFD'].to_numpy()
demand = load_data.loc[:, 'Demand (MW)'].to_numpy()

In [ ]:
# Creating numpy arrays for number of generators, nodes, lines, and time periods. 
B=np.array([g for g in range(0,len(battery_data))]) 
T=np.array([t for t in range(0,len(load_data))])
T1=np.array([t for t in range(1,len(load_data))])

In [ ]:
B #Just as a check

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [ ]:
T #Just as a check

array([0, 1, 2, 3, 4])

In [ ]:
T1 #Just as a check

array([1, 2, 3, 4])

In [ ]:
#ED MODEL
def ED():
    m=ConcreteModel()
    m.dual = Suffix(direction=Suffix.IMPORT)#Create a 'dual' suffix component on the instance so the solver plugin will know which suffixes to collect
    m.B=Set(initialize=B)
    m.T=Set(initialize=T)
    m.T1=Set(initialize=T1)

    #Decision Variables
    m.b=Var(m.B, m.T, within=PositiveReals) # Energy discharged by battery x at time t
    m.e=Var(m.B, m.T, within=PositiveReals) # Energy present in the battery x at time t

    #Objective function
    m.system_cost=Objective(expr=sum(sum(m.b[x,t]*(discharge_unitary_cost[x]) for x in m.B) for t in m.T), sense=minimize)#Objective is to minimize discharging costs

    #Constraints
    # 1. Energy at time t = Energy at time t-1 - discharge
    m.energyconstraint = Constraint(m.B, m.T1, rule=lambda m, x, t1: m.e[x,t1] == m.e[x,t1-1] - m.b[x,t1])
    # 2. Capacity Constraint
    m.capacityconstraint = Constraint(m.B, m.T, rule=lambda m, x, t: m.e[x,t] <= installed_energy_capacity[x])
    # 3. Energy leaving is constrained by discharge capacity
    m.energyleavingconstraint = Constraint(m.B, m.T, rule=lambda m, x, t: m.b[x,t] <= discharge_rate[x]*rated_power[x])
    # 4. Energy Balance Constraint
    m.energybalance = Constraint(m.B, m.T, rule=lambda m, x, t: sum(m.b[x,t]*discharge_efficiency[x] for x in m.B) == demand[t])

    return m

In [ ]:
m = ED()
SolverFactory('cbc', executable='/usr/bin/cbc').solve(m).write() # Solving the model

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 80425.02526
  Upper bound: 80425.02526
  Number of objectives: 1
  Number of constraints: 381
  Number of variables: 201
  Number of nonzeros: 95
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Nu

In [ ]:
m.energyleavingconstraint.pprint() # Just as a check

energyleavingconstraint : Size=100, Index=energyleavingconstraint_index, Active=True
    Key     : Lower : Body    : Upper             : Active
     (0, 0) :  -Inf :  b[0,0] : 66.92999999999999 :   True
     (0, 1) :  -Inf :  b[0,1] : 66.92999999999999 :   True
     (0, 2) :  -Inf :  b[0,2] : 66.92999999999999 :   True
     (0, 3) :  -Inf :  b[0,3] : 66.92999999999999 :   True
     (0, 4) :  -Inf :  b[0,4] : 66.92999999999999 :   True
     (1, 0) :  -Inf :  b[1,0] :             70.81 :   True
     (1, 1) :  -Inf :  b[1,1] :             70.81 :   True
     (1, 2) :  -Inf :  b[1,2] :             70.81 :   True
     (1, 3) :  -Inf :  b[1,3] :             70.81 :   True
     (1, 4) :  -Inf :  b[1,4] :             70.81 :   True
     (2, 0) :  -Inf :  b[2,0] :             94.09 :   True
     (2, 1) :  -Inf :  b[2,1] :             94.09 :   True
     (2, 2) :  -Inf :  b[2,2] :             94.09 :   True
     (2, 3) :  -Inf :  b[2,3] :             94.09 :   True
     (2, 4) :  -Inf :  b[2,4] 

In [ ]:
# Printing the solution
print('SOLUTION')
print('The total system cost is = $',m.system_cost())
print('Commitment and Generation')
for t in T:
 print('At time = '+ str(t))  
 for x in B:
  print('Battery : '+str(x)+','+' Power: '+str(m.b[x,t]()))

SOLUTION
The total system cost is = $ 80425.02526000001
Commitment and Generation
At time = 0
Battery : 0, Power: 60.67
Battery : 1, Power: 70.81
Battery : 2, Power: 94.09
Battery : 3, Power: 49.47
Battery : 4, Power: 54.32
Battery : 5, Power: 80.51
Battery : 6, Power: 0.0
Battery : 7, Power: 0.0
Battery : 8, Power: 56.26
Battery : 9, Power: 0.0
Battery : 10, Power: 79.54
Battery : 11, Power: 70.81
Battery : 12, Power: 54.32
Battery : 13, Power: 0.0
Battery : 14, Power: 53.35
Battery : 15, Power: 82.45
Battery : 16, Power: 65.96
Battery : 17, Power: 65.96
Battery : 18, Power: 81.48
Battery : 19, Power: 0.0
At time = 1
Battery : 0, Power: 66.93
Battery : 1, Power: 70.81
Battery : 2, Power: 94.09
Battery : 3, Power: 49.47
Battery : 4, Power: 54.32
Battery : 5, Power: 0.0
Battery : 6, Power: 72.75
Battery : 7, Power: 51.41
Battery : 8, Power: 56.26
Battery : 9, Power: 86.33
Battery : 10, Power: 1.38
Battery : 11, Power: 14.57
Battery : 12, Power: 54.32
Battery : 13, Power: 0.0
Battery : 1

In [ ]:
# Creating data frame for power flow on line l at time t
output = pd.DataFrame(battery_data["Battery"])
for t in T:
  for x in B:
   output.at[x, str("t = ")+str(t)] = value(m.b[x,t])

In [ ]:
output.to_excel('Battery_Power_Output.xlsx')